通过__tf.data__ API可以从简单，重复使用的pieces中建立输入的pipeline。例如，在image model的广岛输送可能会将数据从分散的文件系统中合并，在每张图片中实现随机的置换并且将随机选择的图片和比你高到一个batch当中用来训练。一个文本模型的管道输送可能会从原始的文本数据当中提取符号，将他们通过一个查阅表来转换成为embedding identifiers。__tf.data__ API让这些数据量大的，不同数据类型和复杂的转换都可以比较简单的处理。

__tf.data__ API为TensorFlow提供了两个抽象类(two abstractions):
* __tf.data.Dataset__ 表示一个元素序列，每一个元素包含了一个或者多个__Tensor__对象。例如，在图片管道中，一个元素可能时一个训练例子，包含由表示图片数据的tensors和标签。这里有两个不同的方法来创建一个dataset:
    * 创建一个源(__source__)(例如，Dataset.from_tensor_slices())从一个或者多个__tf.Tensor__对象中构建一个dataset。
    * 通过转换(__transformation__)(例如, Dataset.batch())从一个或者多个 __tf.data.Dataset__ 对象构建一个dataset。
* __tf.data.Iterator__ 提供了一个从一个dataset中分离元素的主要方法。这个operation通过__Iterator.get_next()__返回，当执行的时候，生成(yield)这个Dataset中的下一个element,一般来说作为输入管道代码和模型的交互。 最简单的迭代是"one-shot iterator"， 与一个特殊的__Dataset__相关并且只迭代一次。 通过更加复杂的(sophisticated)使用，__Iterator.initializer__操作让你可以对不同的dataset重新初始化并且参数化一个iterator。例如，在一个程序当中，对training和validation数据迭代多次。

# Basic mechanics
这一部分描述了怎么样创建不同类型的__Dataset__和__Iterator__对象的基础，并且怎么样从中提取数据。

为了开始一个输入管道(input pipeline)，首先必须定义一个source。例如，可以使用__tf.data.Dataset.from_tensors()或tf.data.Dataset.from_tensor_slices()__从内存中的一些tensor创建一个Dataset。对应的，如果输入数据在磁盘上面则推荐使用TFRecord格式， 你可以创建一个__tf.data.TFRecordDataset__。

一旦你由一个Dataset对象，你可以通过chaining method调用__tf.data.Dataset__对象将其转换成一个新的__Dataset__。例如，你可以应用逐一元素的转换，例如__Dataset.map()__(将一个函数运用到每个元素），或者多元素转换，例如__Dataset.batch()__。查看[tf.data.Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset?hl=zh-cn)文档，查看完整的转换列表。

最常见的从一个Dataset中处理值（consume values）得方式是构造一个__iterator__对象用来每次获得dataset中得一个元素(例如， 通过调用__Dataset.make_one_shot_iterator()__)。 __tf.data.Iterator__提供两个操作： __Iterator.initializer__,可以重新初始化迭代器得状态，__Iterator.get_next()__，用来返回对应得符号下一个元素（symbolic next element)的__tf.Tensor__对象。依赖于使用的情况，选择不同的iterator迭代器，这些选择的介绍如下所示。

## Dataset structure
一个dataset的组成元素都由同样的结构。一个元素包含一个或者多个__tf.Tensor__对象，被称为成分(components)。每个成分由一个__tf.DType__代表tensor当中的元素的类型，__tf.TensorShape__代表（可能是部分指定)每个元素的静态shape。__Dataset.output_types__和__Dataset.output_shapes__特性(properties)让你可以探查在dataset元素中每一个成分的推断types和shapes。这些特性的嵌套结构被映射到一个元素的结构，可能是一个tensor,一个元组的tensor或者一个嵌套元组的tensors。示例如下：

In [2]:
import tensorflow as tf
dataset1 = tf.data.Dataset.from_tensor_slices(tf.random_uniform([4, 10]))
print(dataset1.output_types)  # ==> "tf.float32"
print(dataset1.output_shapes)  # ==> "(10,)"

dataset2 = tf.data.Dataset.from_tensor_slices(
   (tf.random_uniform([4]),
    tf.random_uniform([4, 100], maxval=100, dtype=tf.int32)))
print(dataset2.output_types)  # ==> "(tf.float32, tf.int32)"
print(dataset2.output_shapes)  # ==> "((), (100,))"

dataset3 = tf.data.Dataset.zip((dataset1, dataset2))
print(dataset3.output_types)  # ==> (tf.float32, (tf.float32, tf.int32))
print(dataset3.output_shapes)  # ==> "(10, ((), (100,)))"

<dtype: 'float32'>
(10,)
(tf.float32, tf.int32)
(TensorShape([]), TensorShape([Dimension(100)]))
(tf.float32, (tf.float32, tf.int32))
(TensorShape([Dimension(10)]), (TensorShape([]), TensorShape([Dimension(100)])))


一指定一个元素的每个成分(each component of an element)的名字也是非常方便的,例如，如果他们代表一个training例子的不同的特征。除了元组之外， 你可以使用__collections.nametupe__或者一个字典来将strings映射到tensors来表示Dataset的一个单一的元素，示例如下：

In [6]:
dataset = tf.data.Dataset.from_tensor_slices(
    {
        "a":tf.random_uniform([4]),
        "b":tf.random_uniform([4,100],maxval=100,dtype=tf.int32)
    }
)
print(dataset.output_types)  # ==> "{'a': tf.float32, 'b': tf.int32}"
print(dataset.output_shapes) # ==> "{'a': (), 'b': (100,)}"

{'b': tf.int32, 'a': tf.float32}
{'b': TensorShape([Dimension(100)]), 'a': TensorShape([])}


__Dataset__变换支持任意结构的datasets。 当使用__Dataset.map(), Dataset.flat_map()__和__Dataset.filter()__变换，都是将一个函数运用到每个元素，这个元素的结构决定了函数的参数，示例如下：

In [ ]:
dataset1 = dataset1.map(lambda x:...)

dataset2 = dataset2.map(lambda x, y:...)

# Note: Argument destructuring is not available in Python 3.
dataset3 = dataset3.filter(lambda x, (y,z):...)

## Creating an iterator
一旦你已经创立一个Dataset来表征你的输入数据，下一步就是创造一个Iterator来获得这个dataset中的元素。如果__tf.data__ API现在支持如下的iterators，依照不断增加的复杂程度(increasing level of sophistication):
* one-shot,
* initializable,
* reinitializable, and
* feedable.

__one-shot__ iterator是最简单iterator,支持从dataset中迭代一次，而不需要显式的初始化。 one-shot iterators处理队列基础的输入管道支持(existing queue-based input pipelines support)的几乎所有的情况, 但是他们不支持参数化。 利用下面的 __Dataset.range()__ 示例：
> __Note:__ 现在，one-shot iterators是__Estimator__可以简单使用的唯一类型。

In [8]:
dataset = tf.data.Dataset.range(100)
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

sess = tf.Session()
for i in range(100):
    value = sess.run(next_element)
    assert i==value

__initializable__ iterator 要求在使用之前显式的运行__iterator.initializer__ operation。虽然牺牲了便利性，但是她可以让你参数化dataset的定义，通过使用一个或者多个的__tf.placeholder()__ tensor在初始化iterator的时候作为输入，同样使用__Dataset.range()__的示例：

In [10]:
max_value = tf.placeholder(tf.int64, shape=[])
dataset = tf.data.Dataset.range(max_value)
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()

# Initialize an iterator over a dataset with 10 elements.
sess.run(iterator.initializer, feed_dict={max_value: 10})
for i in range(10):
    value = sess.run(next_element)
    assert i == value
    
# Initialize the same iterator over a dataset with 100 elements.
sess.run(iterator.initializer, feed_dict={max_value: 100})
for i in range(100):
    value = sess.run(next_element)
    assert i == value

__reinitializable__ iterator 可以通过多个不同的__Dataset__对象初始化。例如， 你可能由一个training input pipeline对输入图片使用随机置换来提高泛化能力，validation输入管道在没有修改过的数据上面进行预示评估。 这些管道一般可以使用拥有相同结构的不同Dataset对象(也就是说每个成分由相同的types和可兼容的shapes），示例如下：

In [15]:
# Define training and validation datasets with the same structure.
training_dataset = tf.data.Dataset.range(100).map(
    lambda x: x+tf.random_uniform([],-10,10,tf.int64))
validation_dataset = tf.data.Dataset.range(50)

# A reinitializable iterator is defined by its structure. We could use the
# `output_types` and `output_shapes` properties of either `training_dataset`
# or `validation_dataset` here, because they are compatible.
iterator = tf.data.Iterator.from_structure(training_dataset.output_types,
                                           training_dataset.output_shapes)
next_element = iterator.get_next()

training_init_op = iterator.make_initializer(training_dataset)
validation_init_op = iterator.make_initializer(validation_dataset)

# Run 20 epochs in which the training dataset is traversed, followed by the
# validation dataset.
for _ in range(20):
    # nitialize an iterator over the training dataset.
    sess.run(training_init_op)
    for _ in range(100):
        sess.run(next_element)
        #print(sess.run(next_element))
        
    # Initialize an iterator over the validation dataset.
    sess.run(validation_init_op)
    for _ in range(50):
        sess.run(next_element)
        #print(sess.run(next_element))

7
-7
3
-4
13
12
-4
16
8
14
6
16
6
20
8
19
14
25
17
24
22
27
28
29
21
26
26
36
36
34
22
37
40
27
38
25
28
34
30
35
40
43
49
36
53
45
50
54
42
54
57
41
52
46
58
52
64
51
48
51
63
55
64
65
56
58
59
65
62
76
67
71
78
63
70
67
84
74
69
84
77
82
84
76
76
83
84
79
89
79
95
97
92
99
91
102
86
106
95
89
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
-4
3
6
-7
4
7
12
10
13
9
12
6
3
4
5
14
15
21
21
24
19
17
31
15
22
22
22
34
22
20
33
32
23
35
41
34
34
32
45
33
41
42
34
43
46
51
36
51
40
53
44
53
42
48
63
54
59
62
52
49
50
54
68
62
66
55
62
69
72
71
75
70
65
79
66
70
73
86
86
77
86
73
84
90
76
77
84
81
88
82
80
82
97
95
100
90
105
91
101
93
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
-6
-8
8
-3
11
-3
14
0
7
16
17
6
6
14
18
8
16
18
15
10
13
14
21
24
31
17
34
27
18
26
38
35
32
35
40
30
27
36
40
44
39
49
34
42
39
5

__feedable__ iterator可以和__tf.placeholder__一起用来通过__feed_dict__机制在每次调用__tf.Session.run__的时候选择什么Iterator。它与reinitializable iterator提供了一样的功能，但是这个不要求当你转换不同的iterator的时候需要从dataset的最开始初始化iterator。例如，使用和上面一样的training和validation例子，你可以使用__tf.data.Iterator.from_string_handle__来定义一个feedable iterator让你可以从两个dataset中转换，示例如下：

In [ ]:
# Define training and validation datasets with the same structure.
training_dataset = tf.data.Dataset.range(100).map(
    lambda x: x+tf.random_uniform([],-10,10,tf.int64)).repeat()
validation_dataset = tf.data.Dataset.range(50)

# A feedable iterator is defined by a handle placeholder and its structure. We
# could use the `output_types` and `output_shapes` properties of either
# `training_dataset` or `validation_dataset` here, because they have
# identical structure.
handle = tf.placeholder(tf.string,shape=[])
iterator = tf.data.Iterator.from_string_handle(
    handle, training_dataset.output_types,training_dataset.output_shapes)
next_element = iterator.get_next()

# You can use feedable iterators with a variety of different kinds of iterator
# (such as one-shot and initializable iterators).
training_iterator = training_dataset.make_one_shot_iterator()
validation_iterator = validation_dataset.make_initializable_iterator()

# The `Iterator.string_handle()` method returns a tensor that can be evaluated
# and used to feed the `handle` placeholder.
training_handle = sess.run(training_iterator.string_handle())
validation_handle = sess.run(validation_iterator.string_handle())


# Loop forever, alternating between training and validation.
while True:
    # Run 200 steps using the training dataset. Note that the training dataset is
    # infinite, and we resume from where we left off in the previous `while` loop
    # iteration.
    for _ in range(200):
        sess.run(next_element, feed_dict={handle:training_handle})
        
    # Run one pass over the validation dataset.
    sess.run(validation_iterator.initializer)
    for _ in range(50):
        sess.run(next_element, feed_dict={handle:validation_handle})

## Consuming values from an iterator
__Iterator.get_next()__ 方法返回一个或者多个iterator中symbolic下一个元素对应的tf.Tensor。 每次当这些tensors被评估的时候，他们在潜在的dataset中取得下一个元素的值(注意，就像TensorFlow中的状态对象，调用 __Iterator.get_next()__ 不会马上升级iterator(advance the iterator)。相反你必须要用返回的用TensorFlow表达的tf.Tensor对象，并且将这个结果传递到__tf.Session.run()__获得新的元素并且升级这个iterator)

如果iterator到达了dataset的尾部，执行__Iterator.get_next()__ operation会产生一个__tf.errors.OutOfRangeError__。在这个时候这个iterator会在一个无法使用的状态，如果说还想要使用它就必须再初始化它。  
示例如下：

In [19]:
dataset = tf.data.Dataset.range(5)
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()

# Typically `result` will be the output of a model, or an optimizer's
# training operation.
result = tf.add(next_element,next_element)

sess.run(iterator.initializer)
print(sess.run(result)) # ==> "0"
print(sess.run(result))  # ==> "2"
print(sess.run(result))  # ==> "4"
print(sess.run(result))  # ==> "6"
print(sess.run(result))  # ==> "8"
try:
    sess.run(result)
except tf.errors.OutOfRangeError:
    print("End of dataset")  # ==> "End of dataset"

0
2
4
6
8
End of dataset


一个简单的模式来包装"training loop"式try-except块，代码如下1所示：

如果说dataset中的每个元素都式一个嵌套结构，那么__Iterator.get_next()__的返回值会是同样嵌套结构的一个或者多个__tf.Tensor__对象，代码如下2所示：

注意2代码中的next1，next2，next3对所有的成分都会升级iterator(advance the iterator for all components)。一般的iterator的consumer在一次表达中会包括所有的成分。

In [27]:
# 1:
sess.run(iterator.initializer)
while True:
    try:
        sess.run(result)
    except tf.errors.OutOfRangeError:
        break
        

# 2:
dataset1 = tf.data.Dataset.from_tensor_slices(tf.random_uniform([4,10]))
dataset2 = tf.data.Dataset.from_tensor_slices((tf.random_uniform([4]),
                                             tf.random_uniform([4,100])))
dataset3 = tf.data.Dataset.zip((dataset1,dataset2))

iterator = dataset3.make_initializable_iterator()

sess.run(iterator.initializer)
next1, (next2,next3) = iterator.get_next()
print(sess.run(next1))
print(sess.run(next2))
print(sess.run(next3))

[ 0.1302557   0.54629064  0.93889987  0.42320669  0.89215386  0.1509732
  0.37653673  0.28921282  0.50763071  0.01775301]
0.177905
[ 0.15418148  0.01938093  0.48186696  0.50031829  0.43620992  0.28378272
  0.66841936  0.78226364  0.41728222  0.18975222  0.19912291  0.62860334
  0.3842932   0.25828934  0.09713376  0.93651104  0.14878547  0.20800781
  0.65027189  0.18669283  0.48486602  0.31459033  0.9754014   0.31855345
  0.74826694  0.56987059  0.76902008  0.91922665  0.49015307  0.91039908
  0.93796551  0.67679501  0.92010391  0.63279235  0.57685769  0.99094963
  0.3493154   0.42746437  0.88671494  0.61628211  0.81548679  0.49905503
  0.57373321  0.68571699  0.84236622  0.05715656  0.37960911  0.38795221
  0.89449811  0.99709797  0.90481842  0.90928137  0.64620924  0.47127473
  0.45869982  0.75622976  0.15703666  0.15165269  0.03938413  0.74768841
  0.61789906  0.96700895  0.96673727  0.82380509  0.09683704  0.00919831
  0.3820051   0.51317537  0.32897663  0.09616423  0.92895913  0.40

# Reading input data
## Consuming NumPy arrays
如果所有的input data都在内存中，最简单的方式是将其转换成tf.Tensor对象,再使用使用__Dataset.from_tensor_slices()__，示例如下：

In [ ]:
# Load the training data into two NumPy arrays, for example using `np.load()`.
with np.load("training_data.npy") as data:
    features = data['features']
    labels = data["labels"]
    
# Assume that each row of `features` corresponds to the same row as `labels`.
assert features.shape[0] == labels.shape[0]

dataset = tf.data.Dataset.from_tensor_slice

注意到上面得代码会像__tf.constant()__ operation一样，将feature和labels数列嵌入(embed)到TensorFlow graph当中。这些在小的dataset中作用得很好，但是浪费内存，因为这些array的内容会复制很多次，在tf.GraphDef protocol buffer会运行到2GB的限制。

作为替代者，可以根据__tf.placeholder()__ tensor定义Dataset，当你在dataset初始化__Iterator__时候，传入NumPy arrays。

示例如下：

In [ ]:
# Load the training data into two NumPy arrays, for example using `np.load()`.
with np.load("/var/data/training_data.npy") as data:
    features = data["features"]
    labels = data["labels"]
    
# Assume that each row of `features` corresponds to the same row as `labels`.  
assert features.shape[0] == labels.shape[0]

feature_placeholder = tf.placeholder(feature.dtype, feature.shape)
labels_placeholder = tf.placeholder(labels.dtype, labels.shape)

dataset = tf.data.Dataset.from_tensor_slices((features_placeholder, labels_placeholder))
# [Other transformations on 'dataset'...]
dataset = ...
iterator = dataset.make_initializable_iterator()

sess.run(iterator.initializer,feed_dict={feature_placeholder: features,
                                        labels_placeholder: labels})

## Consuming TFRecord data
__tf.data__ API支持大部分的文件格式，所以可以处理不适用于内存的大的数据量。例如，TFRecord文件格式是一个简单的记录导向的二进制文件，可以被需要TensorFlow的应用用来训练数据。 __tf.data.TFRecordData__ 类让你可以可以让内容流于(stream over)一个或者多个TFRecord文件来作为输入pipeline的一部分，例如：

In [ ]:
# Creates a dataset that reads all of the examples from two files.
filenames = ["/var/data/file1.tfrecord", "/var/data/file2.tfrecord"]
dataset = tf.data.TFRecordDataset(filenames)

在__tf,data.TFRecord__中初始化的__filenames__参数要么是一个字符串，字符串列表，或者字符串的tf.Tensor。因此，如果说由两组文件分别作为training和validation,可以使用__tf.placeholder(tf.string)来表达filenames，从合适的filenames里面来初始化一个iterator，示例如下：

In [ ]:
filenames = tf.placeholder(tf.string, shape=[None])
dataset = tf.data.TFRecordDataset(filenames)
dataset = dataset.map(...)  # Parse the record into tensors
dataset = dataset.repeat()  # Repeat the input indefinitely.
dataset = dataset.batch(32)
iterator = dataset.make_initializable_iterator()

# You can feed the initializer with the appropriate filenames for the current
# phase of execution, e.g. training vs. validation.

# Initialize `iterator` with training data.
training_filenames = ["/var/data/file1.tfrecord", "/var/data/file2.tfrecord"]
sess.run(iterator.initializer, feed_dict={filenames: training_filenames})

# Initialize `iterator` with validation data.
validation_filenames = ["/var/data/validation1.tfrecord", ...]
sess.run(iterator.initializer, feed_dict={filenames: validation_filenames})

## Consuming text data
许多datasets粉鱼与一个或者多个text files。 __tf.data.TextLineDataset__提供了一个简单的方法来从一个或者多个text file中提取lines。给定一个或者多个filenames, 一个__TextLineDataset__可以为这些files的每行创建一个string-valued元素。 类似 __TFRecordDataset，TextLineDataset__ 接收 __tf.Tensor__ 类型的 __filenames__ ，因此你可以通过传递 __tf.placeholder(tf.string)__ 参数化它。

In [ ]:
filenames = ["/var/data/file1.txt", "/var/data/file2.txt"]
dataset = tf.data.TextLineDataset(filenames)

默认情况下， 一个 __TextLineDataset__ 为每个文件产生每一行，但是我们并不是需要每一行，例如一个文件的文件头(header line). 这些行可以通过使用__Dataset.skip()__和__Dataset.filter()__转换可以移除。使用__Dataset.flat_map()__可以为每个file创建一个嵌套的Dataset，用来对每个文件实现不同的转换，示例如下：

In [ ]:
filenames = ["/var/data/file1.txt", "/var/data/file2.txt"]

dataset = tf.data.Dataset.from_tensor_slices(filenames)

# Use `Dataset.flat_map()` to transform each file as a separate nested dataset,
# and then concatenate their contents sequentially into a single "flat" dataset.
# * Skip the first line (header row).
# * Filter out lines beginning with "#" (comments).
dataset = dataset.flat_map(
    lambda filename:(
        tf.data.TextLineDataset(filename)
        .skip(1)
        .filter(lambda line:tf.not_equal(tf.substr(line,0,1),'#'))))

In [ ]:
# substr(input, pos, len, name=None)， 返回子字符串，start at index pos with total length len
input = [b'Hello', b'World'] # b 表示byte string而不是unicode string
position = 1
length = 3

output = [b'ell', b'orl']

# Preprocessing data with Dataset.map()
__Dataset.map(f)__ 转换产生一个新的dataset通过给点一个函数f到input dataset中的每个元素。者是基于map()函数的，在功能性的编程语言当中经常被适用于列表（或者其他结构）。函数f将一个__tf.Tensor__ 对象视为一个输入元素，返回一个__tf.Tensor__ 对象代表新的dataset中的一个元素。 这个实现用标准的TensorFlow operations将一个元素转换成另外一个元素。

这个部分包含了怎么样使用__Dataset.map()__的一般示例

## Parsing tf.Example protocol buffer messages
许多输入管道从一个TFRecord格式文件(写，例如，使用__tf.python_io.TFRecordWriter__ )提取出__tf.train.Example__ 协议缓存信息。 每一个__tf.train.Example__ 记录包含一个或者多个"features",并且输入广澳一般将这些特征转换为tensors。  
示例如下：

In [ ]:
# Transforms a scalar string `example_proto` into a pair of a scalar string and
# a scalar integer, representing an image and its label, respectively.

def _parse_function(example_proto):
    features = {"image": tf.FixedLenFeature((), tf.string, default_value=""),
               "label": tf.FixedLenFeature((), tf.int32, default_value=0)}
    parsed_features = tf.parse_single_example(example_proto,features)
    return parsed_features["image"],parsed_features["label"]

# Creates a dataset that reads all of the examples from two files, and extracts
# the image and label features.
filenames = ["/var/data/file1.tfrecord", "/var/data/file2.tfrecord"]
dataset = tf.data.TFRecordDataset(filenames)
dataset = dataset.map(_parse_function)

## Decoding image data and resizing it
当用image数据训练NN时，经常会将不同的images的尺寸转换到一个普通的尺寸，这样可以将其batched到一个固定的尺寸当中。  
示例如下：

In [ ]:
# Reads an image from a file, decodes it into a dense tensor, and resizes it
# to a fixed shape.
def _parse_function(filename,label):
    image_string = tf.read_file(filename)
    image_decoded = tf.image.decode_image(image_string)
    image_resized = tf.image.resize_images(image_decoded, [28,28])
    return image_resized, label
# A vector of filenames.
filenames = tf.constant(["/var/data/image1.jpg", "/var/data/image2.jpg", ...])

# `labels[i]` is the label for the image in `filenames[i].
labels = tf.constant([0, 37, ...])

dataset = tf.data.Dataset.from_tensor_slices((filenames,labels))
dataset = dataset.map(_parse_function)

## Applying arbitrary Python logic with tf.py_func()
由于性能原因，需要对数据进行额外的预处理，有的时候还需要调用Python的外部库来解析输入的数据。这是，可以激活在__Dataset.map()__转换当中的__tf.py_func()__ operation。  

> py_func(
    func,
    inp,
    Tout,
    stateful=True,
    name=None
)  
Wraps a python function and uses it as a TensorFlow op.  

示例如下：

In [ ]:
import cv2

# Use a custom OpenCV function to read the image, instead of the standard
# TensorFlow `tf.read_file()` operation.
def _read_py_function(filename, label):
    image_decoded = cv2.imread(image_string, cv2.IMREAD_GRAYSCALE)
    return image_decoded, label

# Use standard TensorFlow operations to resize the image to a fixed shape.
def _resize_function(image_decoded,label):
    image_decoded.set_shape([None,None,None])
    image_resized = tf.image.resize_image(image_decoded,[28,28])
    return image_resized,label

filenames = ["/var/data/image1.jpg", "/var/data/image2.jpg", ...]
labels = [0, 37, 29, 1, ...]

dataset = tf.data.Dataset.from_tensor_slice((filenames,labels))
dataset = dataset.map(
    lambda filename,label:tuple(tf.py_func(
    _read_py_function,[filename,label],[tf.uint8,label.dtype])))
dataset = dataset.map(_resize_function)

# Batching dataset elements
## Simple batching
最简单的batching是将一个dataset中n个连续元素堆成一个元素。__Dataset.batch()__ 转换实现了这个功能， 像__tf.stack()__ operator有相同的限制，作用于元素中的每一个成分，也就是说，对每一个成分(component) i,所有的元素必须有同样shape的tensor。  
示例如下：

In [29]:
inc_dataset = tf.data.Dataset.range(100)
dec_dataset = tf.data.Dataset.range(0,-100,-1)
dataset = tf.data.Dataset.zip((inc_dataset, dec_dataset))
batched_dataset = dataset.batch(4)

iterator = batched_dataset.make_one_shot_iterator()
next_element = iterator.get_next()

print(sess.run(next_element))  # ==> ([0, 1, 2,   3],   [ 0, -1,  -2,  -3])
print(sess.run(next_element))  # ==> ([4, 5, 6,   7],   [-4, -5,  -6,  -7])
print(sess.run(next_element))  # ==> ([8, 9, 10, 11],   [-8, -9, -10, -11])

(array([0, 1, 2, 3], dtype=int64), array([ 0, -1, -2, -3], dtype=int64))
(array([4, 5, 6, 7], dtype=int64), array([-4, -5, -6, -7], dtype=int64))
(array([ 8,  9, 10, 11], dtype=int64), array([ -8,  -9, -10, -11], dtype=int64))


## Batching tensors with padding
上面作用于所有的tensor有同样的尺寸。但是，许多模型（例如，序列模型）的输入数据可能有许多不同的尺寸(e.g.不同的序列的长度）。为了解决这种情况，__Dataset.padded_batch()__ 转换可以实现不同shape的tensors可以通过指定一个或者多个维度的填充来实现批量(batch)。  
示例如下： 
> 所用函数：   
fill(
    dims,
    value,
    name=None
)  
padded_batch(
    batch_size,
    padded_shapes,
    padding_values=None
)

__Dataset.padded_batch()__ 转换让你为每个成分的不同的维度设定不同的填充，并且这个是可变长度的（用__None__表示于下面的示例当中）或者固定长度的。他也可以被重写填充值，默认为0。

In [30]:
dataset = tf.data.Dataset.range(100)
dataset = dataset.map(lambda x: tf.fill([tf.cast(x, tf.int32)], x))
dataset = dataset.padded_batch(4,padded_shapes=[None])

iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

print(sess.run(next_element))  # ==> [[0, 0, 0], [1, 0, 0], [2, 2, 0], [3, 3, 3]]
print(sess.run(next_element))  # ==> [[4, 4, 4, 4, 0, 0, 0],
                               #      [5, 5, 5, 5, 5, 0, 0],
                               #      [6, 6, 6, 6, 6, 6, 0],
                               #      [7, 7, 7, 7, 7, 7, 7]]

[[0 0 0]
 [1 0 0]
 [2 2 0]
 [3 3 3]]
[[4 4 4 4 0 0 0]
 [5 5 5 5 5 0 0]
 [6 6 6 6 6 6 0]
 [7 7 7 7 7 7 7]]


# Training workflows
## Processing multiple epochs
__tf.data__ API 提供两个主要方法在同一个data上面处理两个epochs。

在同一个dataset上面通过多个epochs迭代的最简单的方法是使用__Dataset.repeat()__ 转换。示例如下，创建一个dataset并且重复10 epochs的input：

In [ ]:
filenames = ["/var/data/file1.tfrecord", "/var/data/file2.tfrecord"]
dataset = tf.data.TFRecordDataset(filenames)
dataset = dataset.map(...)
dataset = dataset.repeat(10)
dataset = dataset.batch(32)

使用无参数的__Dataset.repeat()__ 转换回重复input不定次。__Dataset.repeat()__ 转换连接它的参数而无须表明一个epoch的结尾和下一个epoch的开始。

如果你想要在每一个epoch接收信号，你可以写一个training循环来捕获在dataset的末尾的__tf.errors.OutOfRangeError__ 。在这个时候，你可能会为每一个epoch收集一些statistic(例如，validation error)。  
示例如下：

In [ ]:
filenames = ["/var/data/file1.tfrecord", "/var/data/file2.tfrecord"]
dataset = tf.data.TFRecordDataset(filenames)
dataset = dataset.map(...)
dataset = dataset.batch(32)
dataset = dataset.make_initializable_iterator()
next_element = iterator.get_next()

# Compute for 100 epoches.
for _ in range(100):
    sess.run(iterator.initializer)
    while True:
        try:
            sess.run(next_element)
        except tf.errors.OutOfRangeError:
            break
    
    # [Perform end-of-epoch calculations here.]

## Randomly shuffling input data
__Dataset.Shuffle()__ 转换 通过一个相似于 __tf.RandomShuffleQueue__ 的算法对输入数据进行随机的洗牌: 它保持了一个固定尺寸的缓存区，并且在这个缓冲区依照均匀概率随机去除下一个元素，示例如下：

> shuffle(  
    buffer_size,  
    seed=None,  
    reshuffle_each_iteration=None #每一次迭代完成是否需要重新洗牌。  
)  
随机shuffles dataset中的元素。

In [ ]:
filenames = ["/var/data/file1.tfrecord", "/var/data/file2.tfrecord"]
dataset = tf.data.TFRecordDataset(filenames)
dataset = dataset.map(...)
dataset = dataset.shuffle(buffer_size=10000)
dataset = dataset.batch(32)
dataset = dataset.repeat()

## Using high-level APIs
__tf.train.MonitoredTrainingSession__ API在很多方面简化了在运行分布设定的TensorFlow。 __MonitoredTrainingSession__ 使用__tf.errors.OutOfRangeError__ 来发送信号表明training已经完成了，所以使用__tf.data__ API，推荐使用 __Dataset.make_one_shot_iterator()__。示例如下：

In [ ]:
filenames = ["/var/data/file1.tfrecord", "/var/data/file2.tfrecord"]
dataset = tf.data.TFRecordDataset(filenames)
dataset = dataset.map(...)
dataset = dataset.shuffle(buffer_size=10000)
dataset = dataset.batch(32)
dataset = dataset.repeat(num_epochs)
iterator = dataset.make_one_shot_iterator()

next_example, next_label = iterator.get_next()
loss = model_function(next_example, next_label)

training_op = tf.train.AdagradOptimizer(...).minimize(loss)

with tf.train.MonitoredTrainingSession(...) as sess:
    while not sess.should_stop():
        sess.run(training_op)

将Dataset作用于__tf.estimator.Estimator__ 的input_fn中，我们也建议使用__Dataset.make_one_shot_iterator()__ 。示例如下：
> class FixedLenFeature  
Configuration for parsing a fixed-length input feature.  
提供一个default_value的值可以将sparse的输入看成dense，否则，解析函数会在有缺失值的例子上失败。  
Fields:  
shape: Shape of input data.  
dtype: Data type of input.  
default_value: Value to be used if an example is missing this feature. It must be compatible with dtype and of the specified shape.

> parse_single_example(  
    serialized, # Tensor标量字符串  
    features, # 字典，将keys映射到FixedLenFeature or VarLenFeature 值.  
    name=None, # 这个操作的名字  
    example_names=None # Tensor的标量字符串，与name相关。
)

In [ ]:
def dataset_input_fn():
    filenames = ["/var/data/file1.tfrecord", "/var/data/file2.tfrecord"]
    dataset = tf.data.TFRecordDataset(filenames)
    
    # Use `tf.parse_single_example()` to extract data from a `tf.Example`
    # protocol buffer, and perform any additional per-record preprocessing.
    def parser(record):
        key_to_feature={
            "image_data": tf.FixedLenFeature((), tf.string, default_value=""),
            "date_time": tf.FixedLenFeature((), tf.int64, default_value=""),
            "label": tf.FixedLenFeature((), tf.int64,
                                    default_value=tf.zeros([], dtype=tf.int64)),
        }
        parsed = tf.parse_single_example(record,keys_to_features)
        
        # Perform dditional preprocessing on the parsed data.
        image = tf.image.decode_jpeg(parsed["image_data"])
        image = tf.reshape(image, [299, 299, 1])
        label = tf.cast(parsed["label"], tf.int32)
        
         return {"image_data": image, "date_time": parsed["date_time"]}, label
        
    # Use `Dataset.map()` to build a pair of a feature dictionary and a label
    # tensor for each example.   
    dataset = dataset.map(parser)
    dataset = dataset.shuffle(buffer_size=10000)
    dataset = dataset.batch(32)
    dataset = dataset.repeat(num_epochs)
    iterator = dataset.make_one_shot_iterator()

    # `features` is a dictionary in which each value is a batch of values for
    # that feature; `labels` is a batch of labels.
    features, labels = iterator.get_next()
    return features, labels    